In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import cv2
import os
import matplotlib.pyplot as plt

# Gerando dados sintéticos de fraudes usando uma GAN

# Função para criar o gerador da GAN
def build_generator(latent_dim):
    model = keras.Sequential([
        layers.Dense(128, activation="relu", input_dim=latent_dim),  # Camada inicial densa com ativação ReLU
        layers.Dense(256, activation="relu"),  # Camada intermediária para aumentar a complexidade
        layers.Dense(512, activation="relu"),  # Mais uma camada para aprendizado mais profundo
        layers.Dense(784, activation="sigmoid"),  # Camada final para gerar saída semelhante a uma imagem 28x28
        layers.Reshape((28, 28, 1))  # Redimensiona a saída para formato de imagem
    ])
    return model

# Função para criar o discriminador da GAN
def build_discriminator():
    model = keras.Sequential([
        layers.Flatten(input_shape=(28, 28, 1)),  # Achata a entrada da imagem para um vetor unidimensional
        layers.Dense(512, activation="relu"),  # Camada densa para extração de padrões
        layers.Dense(256, activation="relu"),  # Camada intermediária
        layers.Dense(1, activation="sigmoid")  # Saída binária (1 para real, 0 para falso)
    ])
    return model

# Definição dos modelos
latent_dim = 100  # Dimensão do vetor de entrada aleatório para o gerador
generator = build_generator(latent_dim)  # Criando o gerador
discriminator = build_discriminator()  # Criando o discriminador
discriminator.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])  # Compilação do discriminador

# Criando a GAN combinando gerador e discriminador
gan_input = keras.Input(shape=(latent_dim,))  # Entrada aleatória
gan_output = discriminator(generator(gan_input))  # Passa pelo gerador e depois pelo discriminador
gan = keras.Model(gan_input, gan_output)
gan.compile(optimizer="adam", loss="binary_crossentropy")  # Compilação da GAN

# Função de treinamento da GAN
def train_gan(generator, discriminator, gan, epochs=1000, batch_size=128):
    for epoch in range(epochs):
        noise = np.random.normal(0, 1, (batch_size, latent_dim))  # Gera amostras aleatórias para entrada do gerador
        generated_data = generator.predict(noise)  # Gera dados sintéticos
        real_data = np.random.normal(0, 1, (batch_size, 28, 28, 1))  # Simulando dados reais com ruído

        labels_real = np.ones((batch_size, 1))  # Rótulos para dados reais (1)
        labels_fake = np.zeros((batch_size, 1))  # Rótulos para dados falsos (0)

        discriminator.train_on_batch(real_data, labels_real)  # Treinamento com dados reais
        discriminator.train_on_batch(generated_data, labels_fake)  # Treinamento com dados falsos

        noise = np.random.normal(0, 1, (batch_size, latent_dim))  # Gera novas amostras aleatórias
        gan.train_on_batch(noise, labels_real)  # Treinamento da GAN tentando enganar o discriminador

# Treina a GAN
train_gan(generator, discriminator, gan)

# Automação da Avaliação de Sinistros com Visão Computacional

def detect_anomalies(image_path):
    """
    Função para detectar anomalias em imagens de sinistros.
    A imagem é processada e analisada pelo discriminador treinado para identificar fraudes.
    """
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Carrega a imagem em escala de cinza
    image = cv2.resize(image, (28, 28))  # Redimensiona a imagem para o formato aceito pelo modelo
    image = image.astype("float32") / 255.0  # Normaliza os pixels para valores entre 0 e 1
    image = np.expand_dims(image, axis=[0, -1])  # Expande as dimensões para compatibilidade com a rede neural

    prediction = discriminator.predict(image)  # Passa a imagem pelo discriminador treinado
    if prediction < 0.5:
        return "Suspeita de fraude detectada"  # Caso o modelo indique que a imagem é falsa
    else:
        return "Imagem consistente com sinistro legítimo"  # Caso a imagem pareça legítima

# Exemplo de uso com uma imagem de sinistro
image_path = "sinistro_teste.jpg"  # Substitua pelo caminho da imagem real
if os.path.exists(image_path):
    resultado = detect_anomalies(image_path)
    print(resultado)
else:
    print("Imagem de teste não encontrada.")  # Caso a imagem não esteja disponível
